# 디비에 크롤링 데이터 삽입하기

table1 = id, path, url, seen, date  
table2 = id, netloc, date  
url = nteloc + path + param  
  
두개의 테이블로 왔다갔다 할 수 있다.  

In [51]:
import sqlite3
con = sqlite3.connect("bot16.db")
cur = con.cursor()

In [52]:
import requests
from html import unescape
from html import escape
from bs4 import BeautifulSoup
import bs4
import time
import json
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}

def download(method ,url, param=None, data=None,headers=None, maxretries=4): ##없을수도 있는 것에 None
    try : 
        resp = requests.request(method, url, params=param, data=data,
                       headers=headers)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and maxretries>0:
            print(maxretries)
            resp = download(method, url, param, data, maxretries-1)
        else:
            print(e.response.status_code)
            print(e.response.reason)
    return resp

In [54]:
cur.executescript("""
    DROP TABLE IF EXISTS table1;
    CREATE TABLE table1(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        table2_id INTEGER NOT NULL,
        path TEXT NOT NULL,
        param TEXT, 
        depth INTEGER NOT NULL,
        inbound INTEGER NOT NULL,
        seen BOOLEAN DEFAULT FALSE,
        date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
    );
    
    DROP TABLE IF EXISTS table2;
    CREATE TABLE table2(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        netloc TEXT NOT NULL,
        date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
    );
""")

In [56]:
url = "https://google.com/search"
html = download("get", url, param={"q":"주지훈"},headers=headers)
dom = BeautifulSoup(html.text, "lxml")

for href in [_.find_parent()["href"]
             for _ in dom.select(".LC20lb")]:
    _urlparse = requests.compat.urlparse(href)
    netloc = "://".join(requests.compat.urlparse(href)[:2])
    cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1",[netloc])
    netlocID = cur.fetchone()
    
    if not netlocID:
        cur.execute("INSERT INTO table2(netloc) VALUES(?)",[netloc])
        
        con.commit()
        
        cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc])
        netlocID = cur.fetchone()
        
    cur.execute("INSERT INTO table1(table2_id, path, param) VALUES(?,?,?)", [netlocID[0], _urlparse[2], _urlparse[4]])
    con.commit()
 #   print(cur.lastrowid, netlocID)

IntegrityError: NOT NULL constraint failed: table1.depth

In [49]:
def parseURL(seed):
    urlList = list()
    
    html = download("get",seed)
    dom = BeautifulSoup(html.text, "lxml")
    
    return [requests.compat.urljoin(url,tag["href"])
            for tag in dom.find_all("a") if tag.has_attr("href") and len(tag["href"])>3]

In [50]:
i = 1
while True:
    cur.execute("""
        SELECT table1.id, table2.netloc, table1.path, table1.param
        FROM table1 
        JOIN table2 
            ON table1.table2_id = table2.id 
        WHERE table1.seen = FALSE 
        ORDER BY table1.date ASC 
        LIMIT 0,1
    """)
    seed = cur.fetchone()
    if not seed or i > 1000:
        break;
    
    i += 1
    cur.execute("""
        UPDATE table1
        SET seen=TRUE
        WHERE id=?
    """, [seed[0]])
    con.commit()
    
    baseURL = "{0}{1}?{2}".format(seed[1], seed[2], seed[3])
    for href in parseURL(baseURL):
        netloc = "://".join(requests.compat.urlparse(href)[:2])
        _urlparse = requests.compat.urlparse(href)
        cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc])
        netlocID = cur.fetchone()
  
        if not netlocID:
                cur.execute("INSERT INTO table2(netloc) VALUES(?)", [netloc])
                con.commit()
                cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc])
                netlocID = cur.fetchone()

        cur.execute("INSERT INTO table1(table2_id, path, param) VALUES (?, ?, ?);", [netlocID[0], _urlparse[2], _urlparse[3]])
        con.commit()
        print(cur.lastrowid, netlocID)


#     print(seed)
#     print("{0}{1}?{2}".format(seed[1], seed[2], seed[3]))
#     break

9 (9,)
10 (9,)
11 (9,)
12 (9,)
13 (9,)
14 (9,)
15 (9,)
16 (9,)
17 (10,)
18 (11,)
19 (9,)
20 (9,)
21 (9,)
22 (9,)
23 (9,)
24 (9,)
25 (9,)
26 (9,)
27 (9,)
28 (9,)
29 (9,)
30 (9,)
31 (9,)
32 (9,)
33 (9,)
34 (9,)
35 (9,)
36 (9,)
37 (9,)
38 (9,)
39 (9,)
40 (9,)
41 (9,)
42 (9,)
43 (9,)
44 (9,)
45 (9,)
46 (9,)
47 (9,)
48 (9,)
49 (9,)
50 (9,)
51 (9,)
52 (9,)
53 (9,)
54 (9,)
55 (9,)
56 (9,)
57 (9,)
58 (9,)
59 (9,)
60 (9,)
61 (9,)
62 (9,)
63 (9,)
64 (9,)
65 (9,)
66 (9,)
67 (9,)
68 (9,)
69 (9,)
70 (9,)
71 (9,)
72 (9,)
73 (9,)
74 (9,)
75 (9,)
76 (9,)
77 (9,)
78 (9,)
79 (9,)
80 (9,)
81 (9,)
82 (9,)
83 (9,)
84 (9,)
85 (9,)
86 (9,)
87 (9,)
88 (9,)
89 (9,)
90 (9,)
91 (9,)
92 (9,)
93 (9,)
94 (9,)
95 (9,)
96 (9,)
97 (9,)
98 (9,)
99 (9,)
100 (9,)
101 (9,)
102 (9,)
103 (9,)
104 (9,)
105 (9,)
106 (9,)
107 (9,)
108 (9,)
109 (9,)
110 (9,)
111 (9,)
112 (9,)
113 (9,)
114 (9,)
115 (9,)
116 (9,)
117 (9,)
118 (9,)
119 (9,)
120 (9,)
121 (9,)
122 (9,)
123 (9,)
124 (9,)
125 (9,)
126 (9,)
127 (9,)
128 (9,)
129 (9,)
1

947 (9,)
948 (3,)
949 (75,)
950 (75,)
951 (9,)
952 (9,)
953 (9,)
954 (9,)
955 (9,)
956 (9,)
957 (9,)
958 (9,)
959 (9,)
960 (9,)
961 (9,)
962 (9,)
963 (3,)
964 (3,)
965 (3,)
966 (3,)
967 (3,)
968 (3,)
969 (3,)
970 (3,)
971 (3,)
972 (3,)
973 (3,)
974 (3,)
975 (3,)
976 (3,)
977 (3,)
978 (3,)
979 (3,)
980 (3,)
981 (3,)
982 (3,)
983 (3,)
984 (3,)
985 (3,)
986 (3,)
987 (3,)
988 (3,)
989 (3,)
990 (3,)
991 (3,)
992 (3,)
993 (3,)
994 (3,)
995 (3,)
996 (3,)
997 (3,)
998 (3,)
999 (3,)
1000 (3,)
1001 (3,)
1002 (3,)
1003 (3,)
1004 (3,)
1005 (3,)
1006 (3,)
1007 (3,)
1008 (3,)
1009 (3,)
1010 (3,)
1011 (3,)
1012 (3,)
1013 (3,)
1014 (3,)
1015 (3,)
1016 (3,)
1017 (3,)
1018 (3,)
1019 (3,)
1020 (3,)
1021 (3,)
1022 (3,)
1023 (3,)
1024 (3,)
1025 (3,)
1026 (3,)
1027 (3,)
1028 (3,)
1029 (3,)
1030 (3,)
1031 (3,)
1032 (3,)
1033 (3,)
1034 (3,)
1035 (3,)
1036 (3,)
1037 (3,)
1038 (3,)
1039 (3,)
1040 (3,)
1041 (3,)
1042 (3,)
1043 (3,)
1044 (3,)
1045 (3,)
1046 (3,)
1047 (3,)
1048 (3,)
1049 (3,)
1050 (3,)
1051 (3,)
1

1745 (171,)
1746 (172,)
1747 (104,)
1748 (85,)
1749 (173,)
1750 (174,)
1751 (175,)
1752 (171,)
1753 (176,)
1754 (9,)
1755 (177,)
1756 (9,)
1757 (9,)
1758 (9,)
1759 (176,)
1760 (9,)
1761 (178,)
1762 (9,)
1763 (9,)
1764 (171,)
1765 (172,)
1766 (104,)
1767 (85,)
1768 (173,)
1769 (174,)
1770 (175,)
1771 (171,)
1772 (176,)
1773 (9,)
1774 (177,)
1775 (9,)
1776 (9,)
1777 (9,)
1778 (176,)
1779 (9,)
1780 (178,)
1781 (9,)
1782 (9,)
404
Not Found
1783 (178,)
404
Not Found
1784 (178,)
404
Not Found
1785 (178,)
404
Not Found
1786 (178,)
404
Not Found
1787 (178,)
404
Not Found
1788 (178,)
4


AttributeError: 'int' object has no attribute 'items'